# 1. Librerías

In [18]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ortools.linear_solver import pywraplp
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df_player_salary= pd.read_csv(r'C:\Users\sfeli\Documents\Upgradehub\DATA\NBA\Datasets Cleaned\salary_2024.csv')

In [34]:
#Seleccionamos las columnas de interés
columns_of_interest = ['FG', 'FGA', 'FG%', '3P', '3PA',
                       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
                       'ORB', 'DRB', 'AST', 'STL', 'BLK', 'PF', 'PTS',
                       'OWS', 'DWS', 'WS', 'Salary']

df_rf = df_player_salary[columns_of_interest]

#Definimos la variable objetivo y las variables predictoras
y = df_rf['Salary']
X = df_rf.drop(['Salary'], axis=1)

#Inicializamos y ajustamos el modelo Random Forest
model = RandomForestRegressor()
model.fit(X, y)

#Obtenemos la importancia de las características
feature_importances = model.feature_importances_
feature_importances_percent = 100.0 * (feature_importances / feature_importances.sum())
indices = np.argsort(feature_importances_percent)[::-1]

#Preparamos datos para la visualización
importance_df = pd.DataFrame({
    'Feature': X.columns[indices],
    'Importance (%)': feature_importances_percent[indices]})

#Creamos el gráfico
fig = px.bar(importance_df, 
             x='Feature', 
             y='Importance (%)',
             title='Importancia de las Características en el Modelo de Random Forest',
             labels={'Feature': 'Características', 'Importance (%)': 'Importancia (%)'},
             color='Importance (%)',
             color_continuous_scale='Viridis',
             template='plotly_dark')

# Configuración adicional del gráfico

fig.update_layout(yaxis_title='', xaxis_title='')
fig.update_xaxes(showgrid=False, gridwidth=0)
fig.update_yaxes(showgrid=False, gridwidth=0)

# Mostrar el gráfico
fig.show()

In [4]:
class Player:
    def __init__(self, name, pos, salary, stats):
        self.name = name
        self.pos = pos
        self.salary = salary
        self.stats = stats

def create_team(df):
    players = {}
    for index, row in df.iterrows():
        stats = {
            'FG': row['FG'], 'FGA': row['FGA'], 'FG%': row['FG%'], '3P': row['3P'], '3PA': row['3PA'],
            '3P%': row['3P%'], '2P': row['2P'], '2PA': row['2PA'], '2P%': row['2P%'], 'FT': row['FT'],
            'FTA': row['FTA'], 'FT%': row['FT%'], 'ORB': row['ORB'], 'DRB': row['DRB'], 'AST': row['AST'],
            'STL': row['STL'], 'BLK': row['BLK'], 'PF': row['PF'], 'PTS': row['PTS'], 'OWS': row['OWS'],
            'DWS': row['DWS'], 'WS': row['WS']
        }
        players[row['Player']] = Player(row['Player'], row['Pos'], row['Salary'], stats)
    return players

def create_solver():
    return pywraplp.Solver.CreateSolver('GLOP')

def define_variables(solver, players):
    return {name: solver.IntVar(0, 1, name) for name in players}

def add_constraints(solver, vars, players, max_salary, min_positions):
    
    #Restricción de presupuesto
    salary_constraint = solver.Constraint(0, max_salary)
    for player_name, player in players.items():
        salary_constraint.SetCoefficient(vars[player_name], player.salary)
    
    #Restricción de número de jugadores
    num_players_constraint = solver.Constraint(10, 14)
    for var in vars.values():
        num_players_constraint.SetCoefficient(var, 1)
    
    #Restricciones de posiciones
    for pos, min_count in min_positions.items():
        pos_constraint = solver.Constraint(min_count, solver.infinity())
        for player_name, player in players.items():
            if player.pos == pos:
                pos_constraint.SetCoefficient(vars[player_name], 1)

def set_objective(solver, vars, players):
    #Definimos los pesos para cada estadística
    weights = {
        'PTS': 18.55, 'FG': 13.52, 'FGA': 9.41, 'FT': 6.76, 'AST': 5.99, 'WS': 5.10, 'OWS': 4.62,
        'FTA': 4.14, 'FT%': 3.97, 'PF': 3.59, '3P%': 3.30, 'STL': 3.15, 'ORB': 2.94, '2PA': 1.99,
        'DRB': 1.94, 'DWS': 1.82, '2P%': 1.71, 'FG%': 1.70, 'BLK': 1.63, '3P': 1.61, '3PA': 1.44,
        '2P': 1.13
    }
    
    objective = solver.Objective()
    for player_name, player in players.items():
        weighted_score = sum(weights[stat] * player.stats[stat] for stat in player.stats)
        objective.SetCoefficient(vars[player_name], weighted_score)
    objective.SetMaximization()

def solve_and_print(solver, vars, players):
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        
        selected_players = [player_name for player_name, var in vars.items() if var.solution_value() == 1]
        total_salary = sum(players[player_name].salary for player_name in selected_players)
        total_stats = {stat: 0 for stat in next(iter(players.values())).stats.keys()}
        
        for player_name in selected_players:
            player = players[player_name]
            for stat in player.stats:
                total_stats[stat] += player.stats[stat]
        
        print(f'Total team salary: ${total_salary}')
        print('Team statistics:')
        for stat, value in total_stats.items():
            print(f' - {stat}: {value}')
        print('Team:')
        for player_name in selected_players:
            player = players[player_name]
            print(f' - {player.name}: {player.pos}, Salary: ${player.salary}')
    else:
        print('An optimal solution was not found')
        

def main():
    
    players = create_team(df_player_salary)
    max_salary = 166626303 #Restricción de presupuesto
    min_positions = {'PF': 2, 'C': 2, 'SG': 2, 'PG': 2, 'SF': 2} #Restricciones de posiciones
    
    solver = create_solver()
    vars = define_variables(solver, players)
    add_constraints(solver, vars, players, max_salary, min_positions)
    set_objective(solver, vars, players)
    solve_and_print(solver, vars, players)

if __name__ == "__main__":
    main()

Total team salary: $148692953.0
Team statistics:
 - FG: 7508.0
 - FGA: 15978.0
 - FG%: 5.65
 - 3P: 1785.0
 - 3PA: 5091.0
 - 3P%: 4.1339999999999995
 - 2P: 5723.0
 - 2PA: 10887.0
 - 2P%: 6.298
 - FT: 3902.0
 - FTA: 4786.0
 - FT%: 9.767999999999999
 - ORB: 896.0
 - DRB: 4322.0
 - AST: 4324.0
 - STL: 988.0
 - BLK: 631.0
 - PF: 1814.0
 - PTS: 20703.0
 - OWS: 41.099999999999994
 - DWS: 35.2
 - WS: 76.4
Team:
 - Paolo Banchero: PF, Salary: $11608080.0
 - Miles Bridges: SF, Salary: $7921300.0
 - Jalen Brunson: PG, Salary: $26346666.0
 - Anthony Edwards: SG, Salary: $13534817.0
 - Shai Gilgeous-Alexander: PG, Salary: $33386850.0
 - Jalen Green: SG, Salary: $9891480.0
 - Tyrese Maxey: PG, Salary: $4343920.0
 - Dejounte Murray: SG, Salary: $18214000.0
 - Alperen Sengun: C, Salary: $3536280.0
 - Cam Thomas: SG, Salary: $2240160.0
 - Franz Wagner: SF, Salary: $5508720.0
 - Victor Wembanyama: C, Salary: $12160680.0
